In [22]:
!pip install openai==0.28
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.1 MB/s eta 0:00:00


# Custom Chatbot Project
TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task

- Dataset used is data retrieved from the Wikipedia API. It is used because not all of the models are trained using New Datasets. Also, it helps the model to give precise answer rather than general one.

In [67]:
import requests
params = {
    "action": "query",
    "prop": "extracts",
    "exlimit": 1,
    "titles": "2024",
    "explaintext": 1,
    "formatversion": 2,
    "format": "json"
}
resp = requests.get("https://en.wikipedia.org/w/api.php", params=params)
response_dict = resp.json()

In [68]:
import  pandas as pd
df = pd.DataFrame()
df['text'] = response_dict['query']['pages'][0]['extract'].split('\n')

In [69]:
df.head()

,text
0,"2024 (MMXXIV) is the current year, and is a le..."
1,"So far, this year has seen the continuation of..."
2,The ongoing Israel–Hamas war has led to spillo...
3,"Approximately 80 countries, representing aroun..."
4,"Among democracies, over 80 percent saw the inc..."


In [70]:
print(len(df))

294


In [71]:
print(df["text"].values[2])

The ongoing Israel–Hamas war has led to spillover into numerous countries, most notably Lebanon, who Israel invaded in October. This followed an intensification of the ongoing conflict between Israel and Hezbollah. In the preceding month, Israel escalated an offensive against the group, including an attack using explosives-laden communication devices, and carrying out a bombing of its headquarters which resulted in the killing of the Secretary-General, Hassan Nasrallah. Ismail Haniyeh, the political leader of Hamas, had already been assassinated in the Iranian capital Tehran in July. The year also saw a rise in activity by the Houthi movement which has contributed to a crisis in the Red Sea that has impacted global shipping. The ongoing conflict has sparked widespread protests against the war.


## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [74]:
from dateutil.parser import parse

# Clean up text to remove empty lines and headings
df = df[(df["text"].str.len() > 0) & (~df["text"].str.startswith("=="))]

# In some cases dates are used as headings instead of being part of the
# text sample; adjust so dated text samples start with dates
prefix = ""
for (i, row) in df.iterrows():
    # If the row already has " - ", it already has the needed date prefix
    if " – " not in row["text"]:
        try:
            # If the row's text is a date, set it as the new prefix
            parse(row["text"])
            prefix = row["text"]
        except:
            # If the row's text isn't a date, add the prefix
            row["text"] = prefix + " – " + row["text"]
df = df[df["text"].str.contains(" – ")].reset_index(drop=True)


In [75]:
df.tail()

,text
200,"Economics – Daron Acemoglu, Simon Johnson and ..."
201,"Literature – Han Kang, for her intense poetic..."
202,"Peace – Nihon Hidankyo, for its efforts to ach..."
203,Physics – John J. Hopfield and Geoffrey E. Hin...
204,Physiology or Medicine – Victor Ambros and Gar...


In [8]:
df["text"].values[2]

' – The ongoing Israel–Hamas war has led to spillover into numerous countries, most notably Lebanon, who Israel invaded in October. This followed an intensification of the ongoing conflict between Israel and Hezbollah. In the preceding month, Israel escalated an offensive against the group, including an attack using explosives-laden communication devices, and carrying out a bombing of its headquarters which resulted in the killing of the Secretary-General, Hassan Nasrallah. Ismail Haniyeh, the political leader of Hamas, had already been assassinated in the Iranian capital Tehran in July. The year also saw a rise in activity by the Houthi movement which has contributed to a crisis in the Red Sea that has impacted global shipping. The ongoing conflict has sparked widespread protests against the war.'

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

In [17]:
import openai
## REplace the API key with "API KEY" before submission
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = "API key"

In [78]:

EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
batch_size = 100
embeddings = []
for i in range(0, len(df), batch_size):
    # Send text data to OpenAI model to get embeddings
    response = openai.Embedding.create(
        input=df.iloc[i:i+batch_size]["text"].tolist(),
        engine=EMBEDDING_MODEL_NAME
    )

    # Add embeddings to list
    embeddings.extend([data["embedding"] for data in response["data"]])

# Add embeddings list to dataframe
df["embeddings"] = embeddings
Original_df = df.copy()

In [79]:
df.head()

,text,embeddings
0,"– 2024 (MMXXIV) is the current year, and is a...","[0.001263580983504653, -0.01789085380733013, -..."
1,"– So far, this year has seen the continuation...","[-0.017639704048633575, -0.010922946035861969,..."
2,– The ongoing Israel–Hamas war has led to spi...,"[-0.036743536591529846, -0.008151800371706486,..."
3,"– Approximately 80 countries, representing ar...","[0.0014809399144724011, -0.020359516143798828,..."
4,"– Among democracies, over 80 percent saw the ...","[-0.018062427639961243, -0.007454856298863888,..."


In [50]:
from openai.embeddings_utils import get_embedding, distances_from_embeddings

def get_rows_sorted_by_relevance(question, df):
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """

    # Get embeddings for the question text
    question_embeddings = get_embedding(question, engine=EMBEDDING_MODEL_NAME)

    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    df_copy = df.copy()
    df_copy["distances"] = distances_from_embeddings(
        question_embeddings,
        df_copy["embeddings"].values,
        distance_metric="cosine"
    )

    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order)
    df_copy.sort_values("distances", ascending=True, inplace=True)
    return df_copy

In [51]:
import tiktoken

def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")

    # Count the number of tokens in the prompt template and question
    prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context:

{}

---

Question: {}
Answer:"""

    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))

    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:

        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count

        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)

### Question 1

In [52]:
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import distances_from_embeddings

In [53]:
q1 = "What are the consequences of the Israel Hammas War?"
emb1 = get_embedding(q1, engine=EMBEDDING_MODEL_NAME)
df["distances_1"] = distances_from_embeddings(
    emb1,
    df['embeddings'].tolist(),
    distance_metric="cosine")
df.sort_values("distances_1", ascending=True, inplace=True)
df.head()


,text,embeddings,distances_1
2,– The ongoing Israel–Hamas war has led to spi...,"[-0.036743536591529846, -0.008151800371706486,...",0.128322
37,February 29 – Israel–Hamas war: Soldiers of th...,"[-0.02552812732756138, 0.0004918270860798657, ...",0.150278
24,January 26 – Israel–Hamas war: The UN's Intern...,"[-0.038509294390678406, -0.006569691002368927,...",0.154578
42,March 14 – The United Nations estimates that a...,"[-0.01658126339316368, -0.00156279350630939, -...",0.168580
96,June 12 – The United Nations' first inquiry in...,"[-0.01897365227341652, -0.015389177948236465, ...",0.171184


In [54]:
df["text"].values[0]

' – The ongoing Israel–Hamas war has led to spillover into numerous countries, most notably Lebanon, who Israel invaded in October. This followed an intensification of the ongoing conflict between Israel and Hezbollah. In the preceding month, Israel escalated an offensive against the group, including an attack using explosives-laden communication devices, and carrying out a bombing of its headquarters which resulted in the killing of the Secretary-General, Hassan Nasrallah. Ismail Haniyeh, the political leader of Hamas, had already been assassinated in the Iranian capital Tehran in July. The year also saw a rise in activity by the Houthi movement which has contributed to a crisis in the Red Sea that has impacted global shipping. The ongoing conflict has sparked widespread protests against the war.'

In [55]:
prompt = create_prompt(q1, df, max_token_count=500)

In [56]:
answer = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    max_tokens=500
)["choices"][0]["text"].strip()

print (answer)

The consequences of the Israel-Hamas war include numerous civilian deaths and injuries, war crimes committed by both sides, and political leaders being assassinated.


### Question 2

In [57]:
q2 = "who was elected as the president of Egypt in 2023?"
emb2 = get_embedding(q2, engine=EMBEDDING_MODEL_NAME)
df["distances_2"] = distances_from_embeddings(
    emb2,
    df['embeddings'].tolist(),
    distance_metric="cosine")
df.sort_values("distances_2", ascending=True, inplace=True)
df.head()

,text,embeddings,distances_1,distances_2
105,June 29 – 2024 Mauritanian presidential electi...,"[-0.0073170349933207035, -0.011719981208443642...",0.263817,0.167586
68,May 6 – 2024 Chadian presidential election: Ma...,"[-0.01163235493004322, -0.014150567352771759, ...",0.258519,0.168872
142,September 7 – 2024 Algerian presidential elect...,"[-0.0047106570564210415, -0.013647757470607758...",0.265975,0.174385
158,October 6 – 2024 Tunisian presidential electio...,"[-0.005716345738619566, -0.014978109858930111,...",0.260174,0.175280
45,March 24 – 2024 Senegalese presidential electi...,"[-0.014318631961941719, -0.007357830181717873,...",0.274388,0.176126


In [58]:
df["text"].values[0]

'June 29 – 2024 Mauritanian presidential election: Incumbent president Mohamed Ould Ghazouani wins re-election to a second term.'

In [59]:
prompt = create_prompt(q2, df, max_token_count=500)
answer = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    max_tokens=500
)["choices"][0]["text"].strip()

print (answer)

I don't know.


As Could be see, the completion model has no access to recent information as the Election result of the Egyptian president selected in 2023.
However, The dataset we have calculated the closest answer to the question and it was wrong. We can do something cool here. We can add this piece of information to the data frame.


In [80]:
emb2 = get_embedding(q2, engine=EMBEDDING_MODEL_NAME)
df2 = Original_df
df2.loc[len(df2)] = {'embeddings':emb2,
                   'text': 'In December 2023, Egypt held its presidential election, resulting in the re-election of incumbent President Abdel Fattah el-Sissi for a third six-year term. The National Election Authority announced that el-Sissi secured 89.6% of the vote, with a voter turnout of 66.8%, marking the highest participation rate in Egypt electoral history.  His closest competitor, Hazem Omar, received 4.5% of the votes. This election took place amid significant economic challenges, including surging inflation and a depreciating currency, which have intensified public dissatisfaction.  Despite these issues, el-Sissi substantial victory indicates continued support for his leadership.'
                   }

In [81]:
df2.tail()

,text,embeddings
201,"Literature – Han Kang, for her intense poetic...","[0.0046767196618020535, -0.008849996142089367,..."
202,"Peace – Nihon Hidankyo, for its efforts to ach...","[-0.015292845666408539, -0.011684050783514977,..."
203,Physics – John J. Hopfield and Geoffrey E. Hin...,"[-0.001915285363793373, 0.01271437294781208, 0..."
204,Physiology or Medicine – Victor Ambros and Gar...,"[-0.015671782195568085, -0.012634165585041046,..."
205,"In December 2023, Egypt held its presidential ...","[-0.011872970499098301, -0.031616710126399994,..."


In [85]:
q2 = "who was elected as the president of Egypt in 2023?"
emb2 = get_embedding(q2, engine=EMBEDDING_MODEL_NAME)
df2["distances_2"] = distances_from_embeddings(
    emb2,
    df2['embeddings'].tolist(),
    distance_metric="cosine")
df2.sort_values("distances_2", ascending=True, inplace=True)
df2.head()

,text,embeddings,distances_2
205,"In December 2023, Egypt held its presidential ...","[-0.011872970499098301, -0.031616710126399994,...",0.000000
105,June 29 – 2024 Mauritanian presidential electi...,"[-0.007335253059864044, -0.01173767726868391, ...",0.167626
68,May 6 – 2024 Chadian presidential election: Ma...,"[-0.01163235493004322, -0.014150567352771759, ...",0.168872
142,September 7 – 2024 Algerian presidential elect...,"[-0.004726119805127382, -0.013671760447323322,...",0.174298
158,October 6 – 2024 Tunisian presidential electio...,"[-0.0057285744696855545, -0.014989341609179974...",0.175354


In [87]:
print(df2["text"].values[0])

In December 2023, Egypt held its presidential election, resulting in the re-election of incumbent President Abdel Fattah el-Sissi for a third six-year term. The National Election Authority announced that el-Sissi secured 89.6% of the vote, with a voter turnout of 66.8%, marking the highest participation rate in Egypt electoral history.  His closest competitor, Hazem Omar, received 4.5% of the votes. This election took place amid significant economic challenges, including surging inflation and a depreciating currency, which have intensified public dissatisfaction.  Despite these issues, el-Sissi substantial victory indicates continued support for his leadership.


In [88]:
prompt = create_prompt(q2, df2, max_token_count=500)
answer = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    max_tokens=500
)["choices"][0]["text"].strip()

print (answer)

Incumbent President Abdel Fattah el-Sissi was re-elected as the president of Egypt in 2023.


As you can see, we were able to achieve the goal of getting the correct answer from a completion model by providing new dataset that the model hasn't seen before